Join the data from Part 1 with the data from Part 2 to create a new dataframe.

In [129]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline     
sns.set(color_codes=True)
# import requests
import datetime as dt
import calendar

In [130]:
# Import data from file
df_ctybks = pd.read_csv('data/df_citybikes.csv')
df_fsq = pd.read_csv('data/df_fsq_all_loci102.csv')
df_yelp = pd.read_csv('data/df_yelp_all_loci102.csv')

In [131]:
print(df_ctybks.shape)
print(df_fsq.shape)

(241, 10)
(2410, 15)


Provide a visualization that you used as part of your EDA process. Explain the initial pattern or relationship you discoved through this visualization. 

In [132]:
# Create a new column with the coordinate data we need, reformated to str dtype.

# Grab just the two columns we need lat and lon
df_ctybks.dropna(subset=['latitude', 'longitude'], inplace=True)

# Creation of a new variable with lon and lat together, label the column header 'lat-long'
df_ctybks['lat-long'] = df_ctybks['latitude'].astype(str) + ',' + df_ctybks['longitude'].astype(str)
df_ctybks = df_ctybks[df_ctybks['lat-long'] != '0.0,0.0']
df_ctybks.head()

,Station Name,slots,empty_slots,free_bikes,status,bank_card,latitude,longitude,timestamp,Unique ID,lat-long
0,0001 10th & Cambie,35,16,19,online,No,49.262487,-123.114397,2022-11-06T07:22:52.004000Z,1,"49.262487,-123.114397"
1,0004 Yaletown-Roundhouse Station,16,1,15,online,No,49.274566,-123.121817,2022-11-06T07:22:52.006000Z,4,"49.274566,-123.121817"
2,0005 Dunsmuir & Beatty,26,8,18,online,No,49.279764,-123.110154,2022-11-06T07:22:52.007000Z,5,"49.279764,-123.110154"
3,0007 12th & Yukon (City Hall),16,9,7,online,No,49.260599,-123.113504,2022-11-06T07:22:52.008000Z,7,"49.260599,-123.113504"
4,0008 8th & Ash,16,9,7,online,No,49.264215,-123.117772,2022-11-06T07:22:52.009000Z,8,"49.264215,-123.117772"


In [133]:
# Split the Station id number from the list of station names:
st_id = df_ctybks['Station Name']
id_nums = []
st_names = []
for id in st_id:
    if id[:4].isnumeric():
        id_num = id[:4]
        st_name = id[4:]
    elif id[:3].isnumeric():
        id_num = id[:3]
        st_name = id[:3]
    else: None
    num = int(id_num)

    st_names.append(st_name)
    id_nums.append(num)

# Confirm total 241 values:
print(len(id_nums))
print(len(st_names))
# Confirm type change to int
print(type(id_nums[0]))
print(type(st_names[0]))
# Check if the ids are unique
print(len(id_nums)==len(set(id_nums)))
print(len(st_names)==len(set(st_names)))
# Display a sample
print(id_nums[:14])
print(st_names[:14])

241
241
<class 'int'>
<class 'str'>
True
True
[1, 4, 5, 7, 8, 9, 10, 11, 14, 15, 16, 17, 19, 21]
[' 10th & Cambie', ' Yaletown-Roundhouse Station', ' Dunsmuir & Beatty', ' 12th & Yukon (City Hall)', ' 8th & Ash', ' Spyglass & Seawall', ' Stamps Landing', ' Ontario & Seawall', ' Canada Place', ' Granville & Georgia', ' Pender & Burrard', ' Marinaside & Davie', ' Robson & Hornby', ' Cordova & Granville']


In [134]:
# Add the new columns into the dataframe and confirm their addition
df_ctybks.insert(1, 'st_id', id_nums, False)
df_ctybks.insert(2, 'station_name', st_names, False)
df_ctybks.head(5)

,Station Name,st_id,station_name,slots,empty_slots,free_bikes,status,bank_card,latitude,longitude,timestamp,Unique ID,lat-long
0,0001 10th & Cambie,1,10th & Cambie,35,16,19,online,No,49.262487,-123.114397,2022-11-06T07:22:52.004000Z,1,"49.262487,-123.114397"
1,0004 Yaletown-Roundhouse Station,4,Yaletown-Roundhouse Station,16,1,15,online,No,49.274566,-123.121817,2022-11-06T07:22:52.006000Z,4,"49.274566,-123.121817"
2,0005 Dunsmuir & Beatty,5,Dunsmuir & Beatty,26,8,18,online,No,49.279764,-123.110154,2022-11-06T07:22:52.007000Z,5,"49.279764,-123.110154"
3,0007 12th & Yukon (City Hall),7,12th & Yukon (City Hall),16,9,7,online,No,49.260599,-123.113504,2022-11-06T07:22:52.008000Z,7,"49.260599,-123.113504"
4,0008 8th & Ash,8,8th & Ash,16,9,7,online,No,49.264215,-123.117772,2022-11-06T07:22:52.009000Z,8,"49.264215,-123.117772"


In [135]:
# Remove old station name column
df_ctybks = df_ctybks.drop(['Station Name'], axis = 1)
df_ctybks.head()

,st_id,station_name,slots,empty_slots,free_bikes,status,bank_card,latitude,longitude,timestamp,Unique ID,lat-long
0,1,10th & Cambie,35,16,19,online,No,49.262487,-123.114397,2022-11-06T07:22:52.004000Z,1,"49.262487,-123.114397"
1,4,Yaletown-Roundhouse Station,16,1,15,online,No,49.274566,-123.121817,2022-11-06T07:22:52.006000Z,4,"49.274566,-123.121817"
2,5,Dunsmuir & Beatty,26,8,18,online,No,49.279764,-123.110154,2022-11-06T07:22:52.007000Z,5,"49.279764,-123.110154"
3,7,12th & Yukon (City Hall),16,9,7,online,No,49.260599,-123.113504,2022-11-06T07:22:52.008000Z,7,"49.260599,-123.113504"
4,8,8th & Ash,16,9,7,online,No,49.264215,-123.117772,2022-11-06T07:22:52.009000Z,8,"49.264215,-123.117772"


In [136]:
df_ctybks.tail()

,st_id,station_name,slots,empty_slots,free_bikes,status,bank_card,latitude,longitude,timestamp,Unique ID,lat-long
236,516,Heather & 29th,22,21,1,online,No,49.245535,-123.120496,2022-11-06T07:22:52.164000Z,516,"49.245535,-123.120496"
237,41,Cardero & Robson,26,0,26,online,No,49.289255,-123.132677,2022-11-06T07:22:52.025000Z,41,"49.289255,-123.132677"
238,352,Commercial & 20th,18,15,3,online,No,49.252656,-123.067965,2022-11-06T07:22:52.152000Z,352,"49.252656,-123.067965"
239,196,Hornby & Drake,24,9,15,online,No,49.277527,-123.129464,2022-11-06T07:22:52.105000Z,196,"49.277527,-123.129464"
240,210,210,18,10,8,online,No,49.259469,-123.145718,2022-11-06T07:22:52.172000Z,210,"49.259469,-123.145718"


In [137]:
# Before merging, the lat & long columns can become confusing, so rename them first
df_ctybks = df_ctybks.rename(columns = {'latitude':'st_lat', 'longitude':'st_long', 'free_bikes': 'available'})
df_ctybks.head(3)

,st_id,station_name,slots,empty_slots,available,status,bank_card,st_lat,st_long,timestamp,Unique ID,lat-long
0,1,10th & Cambie,35,16,19,online,No,49.262487,-123.114397,2022-11-06T07:22:52.004000Z,1,"49.262487,-123.114397"
1,4,Yaletown-Roundhouse Station,16,1,15,online,No,49.274566,-123.121817,2022-11-06T07:22:52.006000Z,4,"49.274566,-123.121817"
2,5,Dunsmuir & Beatty,26,8,18,online,No,49.279764,-123.110154,2022-11-06T07:22:52.007000Z,5,"49.279764,-123.110154"


In [138]:
# # Before merging, the lat & long columns can become confusing, so remove them first
# df_ctybks = df_ctybks.drop(['latitude', 'longitude', 'Unique ID'], axis = 1)
# df_ctybks.head(5)

In [139]:
# Does 'bank_card' or 'status' provide any meaningful data?
print(df_ctybks['bank_card'].unique())
print(df_ctybks['status'].unique())

['No']
['online' 'offline']


In [140]:
# Removing 'bank_card' and 'Unique ID'
df_ctybks = df_ctybks.drop(['bank_card', 'Unique ID'], axis = 1)
df_ctybks.head(5)

,st_id,station_name,slots,empty_slots,available,status,st_lat,st_long,timestamp,lat-long
0,1,10th & Cambie,35,16,19,online,49.262487,-123.114397,2022-11-06T07:22:52.004000Z,"49.262487,-123.114397"
1,4,Yaletown-Roundhouse Station,16,1,15,online,49.274566,-123.121817,2022-11-06T07:22:52.006000Z,"49.274566,-123.121817"
2,5,Dunsmuir & Beatty,26,8,18,online,49.279764,-123.110154,2022-11-06T07:22:52.007000Z,"49.279764,-123.110154"
3,7,12th & Yukon (City Hall),16,9,7,online,49.260599,-123.113504,2022-11-06T07:22:52.008000Z,"49.260599,-123.113504"
4,8,8th & Ash,16,9,7,online,49.264215,-123.117772,2022-11-06T07:22:52.009000Z,"49.264215,-123.117772"


In [141]:
# Convert the string format to datetime object
df_ctybks['timestamp'] = pd.to_datetime(df_ctybks['timestamp'], format='%Y%m%d %H:%M')
# Remove time and hour values from timestamp and store in new columns
df_ctybks['date_cb'] = df_ctybks['timestamp'].dt.date
df_ctybks['hour_cb'] = df_ctybks['timestamp'].dt.hour
# Display the dataframe
df_ctybks.head()

,st_id,station_name,slots,empty_slots,available,status,st_lat,st_long,timestamp,lat-long,date_cb,hour_cb
0,1,10th & Cambie,35,16,19,online,49.262487,-123.114397,2022-11-06 07:22:52.004000+00:00,"49.262487,-123.114397",2022-11-06,7
1,4,Yaletown-Roundhouse Station,16,1,15,online,49.274566,-123.121817,2022-11-06 07:22:52.006000+00:00,"49.274566,-123.121817",2022-11-06,7
2,5,Dunsmuir & Beatty,26,8,18,online,49.279764,-123.110154,2022-11-06 07:22:52.007000+00:00,"49.279764,-123.110154",2022-11-06,7
3,7,12th & Yukon (City Hall),16,9,7,online,49.260599,-123.113504,2022-11-06 07:22:52.008000+00:00,"49.260599,-123.113504",2022-11-06,7
4,8,8th & Ash,16,9,7,online,49.264215,-123.117772,2022-11-06 07:22:52.009000+00:00,"49.264215,-123.117772",2022-11-06,7


In [142]:
# Confirm the datetime object data change
df_ctybks.dtypes

st_id                         int64
station_name                 object
slots                         int64
empty_slots                   int64
available                     int64
status                       object
st_lat                      float64
st_long                     float64
timestamp       datetime64[ns, UTC]
lat-long                     object
date_cb                      object
hour_cb                       int64
dtype: object

In [144]:
print(df_fsq.dtypes)
df_fsq.head(3)

lat-long          object
fsq_id            object
cat_id           float64
category_name     object
categories        object
name              object
distance           int64
latitude         float64
longitude        float64
address           object
popularity       float64
price            float64
open_now            bool
rating           float64
rating_count     float64
dtype: object


,lat-long,fsq_id,cat_id,category_name,categories,name,distance,latitude,longitude,address,popularity,price,open_now,rating,rating_count
0,"49.262487,-123.114397",4fd3a01be4b0540d9cb027ea,13034.0,Café,"[{'id': 13034, 'name': 'Café', 'icon': {'prefi...",Moii Cafe,57,49.262562,-123.115171,"2559 Cambie St (btwn W Broadway & W 10th Ave),...",0.221702,1.0,False,8.0,55.0
1,"49.262487,-123.114397",4b1db335f964a520211424e3,13034.0,Café,"[{'id': 13034, 'name': 'Café', 'icon': {'prefi...",Marulilu Cafe,96,49.263364,-123.113942,"451W Broadway W (at Cambie St), Vancouver BC V...",0.953849,2.0,False,8.2,111.0
2,"49.262487,-123.114397",4db0b65d4df03036e8c0960c,11064.0,Hair Salon,"[{'id': 11064, 'name': 'Hair Salon', 'icon': {...",Space Salon,128,49.263102,-123.115785,"530W Broadway W (btw Ash & Cambie), Vancouver ...",0.785993,NaN,False,8.3,12.0


In [145]:
print(df_yelp.dtypes)

lat-long          object
category_name     object
categories        object
name              object
distance         float64
latitude         float64
longitude        float64
address           object
price             object
is_closed           bool
rating           float64
rating_count       int64
dtype: object


now for the foursquare dataframe:

In [146]:
print(df_fsq.shape)
print(df_yelp.shape)
print(df_ctybks.shape)

(2410, 15)
(4788, 12)
(241, 12)


In [147]:
df_merged = pd.merge(df_ctybks, df_fsq, how='inner', on='lat-long')
print(df_merged.shape)
df_merged.head(2)

(2410, 26)


,st_id,station_name,slots,empty_slots,available,status,st_lat,st_long,timestamp,lat-long,...,name,distance,latitude,longitude,address,popularity,price,open_now,rating,rating_count
0,1,10th & Cambie,35,16,19,online,49.262487,-123.114397,2022-11-06 07:22:52.004000+00:00,"49.262487,-123.114397",...,Moii Cafe,57,49.262562,-123.115171,"2559 Cambie St (btwn W Broadway & W 10th Ave),...",0.221702,1.0,False,8.0,55.0
1,1,10th & Cambie,35,16,19,online,49.262487,-123.114397,2022-11-06 07:22:52.004000+00:00,"49.262487,-123.114397",...,Marulilu Cafe,96,49.263364,-123.113942,"451W Broadway W (at Cambie St), Vancouver BC V...",0.953849,2.0,False,8.2,111.0


In [148]:
# Checking the types of data
df = df_merged.copy()
df.dtypes

st_id                          int64
station_name                  object
slots                          int64
empty_slots                    int64
available                      int64
status                        object
st_lat                       float64
st_long                      float64
timestamp        datetime64[ns, UTC]
lat-long                      object
date_cb                       object
hour_cb                        int64
fsq_id                        object
cat_id                       float64
category_name                 object
categories                    object
name                          object
distance                       int64
latitude                     float64
longitude                    float64
address                       object
popularity                   float64
price                        float64
open_now                        bool
rating                       float64
rating_count                 float64
dtype: object

In [155]:
# Create the New Tables

# Bike users table: 
'''
potentially relevant data for a citybik.es user, accessing data for 
or at any particular location of a citybik.es station
'''
Citybikes_users = df[[
    'st_id'
    , 'station_name'
    , 'slots'
    , 'empty_slots'
    , 'available'
    , 'status'
    , 'st_lat'
    , 'st_long'
    , 'timestamp'
    , 'date_cb'
    , 'hour_cb'
    , 'fsq_id'
]]

Citybikes_datetime = df[[
    'st_id'
     , 'station_name'
     , 'fsq_id'
     , 'timestamp'
     , 'date_cb'
     , 'hour_cb'
     ]]

POI_for_users = df[[
    'st_id'
    , 'station_name'
    , 'fsq_id'
    , 'cat_id'
    , 'category_name'
    , 'name'
    , 'distance'
    , 'latitude'
    , 'longitude'
    , 'open_now'
    ]]

Media = df[[
    'st_id'
    , 'station_name'
    , 'fsq_id'
    , 'cat_id'
    , 'category_name'
    , 'name'
    , 'popularity'
    , 'price'
    , 'rating'
    , 'rating_count'
    , 'distance'
    , 'latitude'
    , 'longitude'
    , 'open_now'
    ]]
 
Business = df[[
    'st_id'
     , 'station_name'
     , 'fsq_id'
     , 'cat_id'
     , 'category_name'
     , 'categories'
     , 'name'
     , 'address'
     , 'distance'
     , 'latitude'
     , 'longitude'
     , 'open_now'
     ]]

In [158]:
Citybikes_users.head()

,st_id,station_name,slots,empty_slots,available,status,st_lat,st_long,timestamp,date_cb,hour_cb,fsq_id
0,1,10th & Cambie,35,16,19,online,49.262487,-123.114397,2022-11-06 07:22:52.004000+00:00,2022-11-06,7,4fd3a01be4b0540d9cb027ea
1,1,10th & Cambie,35,16,19,online,49.262487,-123.114397,2022-11-06 07:22:52.004000+00:00,2022-11-06,7,4b1db335f964a520211424e3
2,1,10th & Cambie,35,16,19,online,49.262487,-123.114397,2022-11-06 07:22:52.004000+00:00,2022-11-06,7,4db0b65d4df03036e8c0960c
3,1,10th & Cambie,35,16,19,online,49.262487,-123.114397,2022-11-06 07:22:52.004000+00:00,2022-11-06,7,4bc290d674a9a593788ed3f6
4,1,10th & Cambie,35,16,19,online,49.262487,-123.114397,2022-11-06 07:22:52.004000+00:00,2022-11-06,7,4e223f06d4c0d32590f80ff4


In [159]:
Citybikes_datetime.head()

,st_id,station_name,fsq_id,timestamp,date_cb,hour_cb
0,1,10th & Cambie,4fd3a01be4b0540d9cb027ea,2022-11-06 07:22:52.004000+00:00,2022-11-06,7
1,1,10th & Cambie,4b1db335f964a520211424e3,2022-11-06 07:22:52.004000+00:00,2022-11-06,7
2,1,10th & Cambie,4db0b65d4df03036e8c0960c,2022-11-06 07:22:52.004000+00:00,2022-11-06,7
3,1,10th & Cambie,4bc290d674a9a593788ed3f6,2022-11-06 07:22:52.004000+00:00,2022-11-06,7
4,1,10th & Cambie,4e223f06d4c0d32590f80ff4,2022-11-06 07:22:52.004000+00:00,2022-11-06,7


In [160]:
POI_for_users.head()

,st_id,station_name,fsq_id,cat_id,category_name,name,distance,latitude,longitude,open_now
0,1,10th & Cambie,4fd3a01be4b0540d9cb027ea,13034.0,Café,Moii Cafe,57,49.262562,-123.115171,False
1,1,10th & Cambie,4b1db335f964a520211424e3,13034.0,Café,Marulilu Cafe,96,49.263364,-123.113942,False
2,1,10th & Cambie,4db0b65d4df03036e8c0960c,11064.0,Hair Salon,Space Salon,128,49.263102,-123.115785,False
3,1,10th & Cambie,4bc290d674a9a593788ed3f6,11046.0,Banking and Finance,Vancity Savings Credit Union,48,49.262392,-123.115157,False
4,1,10th & Cambie,4e223f06d4c0d32590f80ff4,13306.0,Taco Restaurant,La Taqueria Pinche Taco Shop,164,49.263601,-123.112665,True


In [161]:
Media.head()

,st_id,station_name,fsq_id,cat_id,category_name,name,popularity,price,rating,rating_count,distance,latitude,longitude,open_now
0,1,10th & Cambie,4fd3a01be4b0540d9cb027ea,13034.0,Café,Moii Cafe,0.221702,1.0,8.0,55.0,57,49.262562,-123.115171,False
1,1,10th & Cambie,4b1db335f964a520211424e3,13034.0,Café,Marulilu Cafe,0.953849,2.0,8.2,111.0,96,49.263364,-123.113942,False
2,1,10th & Cambie,4db0b65d4df03036e8c0960c,11064.0,Hair Salon,Space Salon,0.785993,NaN,8.3,12.0,128,49.263102,-123.115785,False
3,1,10th & Cambie,4bc290d674a9a593788ed3f6,11046.0,Banking and Finance,Vancity Savings Credit Union,0.944987,NaN,7.2,8.0,48,49.262392,-123.115157,False
4,1,10th & Cambie,4e223f06d4c0d32590f80ff4,13306.0,Taco Restaurant,La Taqueria Pinche Taco Shop,0.954043,2.0,8.7,367.0,164,49.263601,-123.112665,True


In [162]:
Business.head()

,st_id,station_name,fsq_id,cat_id,category_name,categories,name,address,distance,latitude,longitude,open_now
0,1,10th & Cambie,4fd3a01be4b0540d9cb027ea,13034.0,Café,"[{'id': 13034, 'name': 'Café', 'icon': {'prefi...",Moii Cafe,"2559 Cambie St (btwn W Broadway & W 10th Ave),...",57,49.262562,-123.115171,False
1,1,10th & Cambie,4b1db335f964a520211424e3,13034.0,Café,"[{'id': 13034, 'name': 'Café', 'icon': {'prefi...",Marulilu Cafe,"451W Broadway W (at Cambie St), Vancouver BC V...",96,49.263364,-123.113942,False
2,1,10th & Cambie,4db0b65d4df03036e8c0960c,11064.0,Hair Salon,"[{'id': 11064, 'name': 'Hair Salon', 'icon': {...",Space Salon,"530W Broadway W (btw Ash & Cambie), Vancouver ...",128,49.263102,-123.115785,False
3,1,10th & Cambie,4bc290d674a9a593788ed3f6,11046.0,Banking and Finance,"[{'id': 11046, 'name': 'Banking and Finance', ...",Vancity Savings Credit Union,"501 10th Ave W, Vancouver BC V5Z 1K9",48,49.262392,-123.115157,False
4,1,10th & Cambie,4e223f06d4c0d32590f80ff4,13306.0,Taco Restaurant,"[{'id': 13306, 'name': 'Taco Restaurant', 'ico...",La Taqueria Pinche Taco Shop,"2450 Yukon St (at W Broadway), Vancouver BC V5...",164,49.263601,-123.112665,True


In [32]:
# Dropping irrelevant columns
# df = df.drop(['categories', 'Unique ID'], axis=1)

In [31]:
# Recheck headings:
# df.dtypes

Unnamed: 0         int64
Station Name      object
slots              int64
empty_slots        int64
free_bikes         int64
status            object
bank_card         object
latitude_x       float64
longitude_x      float64
timestamp         object
lat-long          object
fsq_id            object
cat_id           float64
category_name     object
name              object
distance           int64
latitude_y       float64
longitude_y      float64
address           object
popularity       float64
price            float64
open_now            bool
rating           float64
rating_count     float64
dtype: object

In [ ]:
# Save tables to data file

In [ ]:
# Visualization things

In [ ]:
# SQLite DB creation

In [ ]:
# An example of groupby:
# df.groupby(df.server_id).resample('M')['cpu_utilization', 'free_memory'].max()

In [ ]:
# Renaming the columns if needed - say if you wanted them "short and pithy Dr B!"
# df = df.rename(columns = { 'Lengthy_current':'LP', 'doggy_booty_toot':'Lucy_etc.' })
# df.head(5)

In [ ]:
# Checking for the presence and location of duplicate rows
# duplicate_rows_df = df[df.duplicated()]

# And compare the difference in the DF shape:
# print(df.shape)
# print("number of duplicate rows: ", duplicate_rows_df.shape)

In [ ]:
# Assess the location and content of the duplications, are they appropriate?
# If not, we can drop them:

# First a count of the total rows to compare to #of duplicate rows
# df.count()

# Then we drop, and recheck:
# df = df.drop_duplicates()
# df.head(5)

In [ ]:
# Drop any missing or null values:
# print(df.isnull().sum())

In [ ]:
# We'll drop the full rows that contain null values:
# df = dfdropna()
# df.count()

In [ ]:
# Validating the dropped missing values
# print(df.isnull().sum())

In [ ]:
# Detecting outliers on numerical values
# sns.boxplot(x=df['col_with_numerical_value'])

# Database

Put all your results in an SQLite3 database (remember, SQLite stores its databases as files in your local machine - make sure to create your database in your project's data/ directory!)

In [ ]:
# Jamie's reuseable library of database functions

# NOTE: Using SQLite in this example, but can follow a similar workflow to connect 
# to any other database when we have the correct Python library:
# `mysql-connector-python` for MySQL
# `psycopg2` for Postgres
# `SQLAlchemy` - generic library which using JDBC to connect to different DBs
import sqlite3
from sqlite3 import Error

def create_connection(path):
    connection = None
    try:
        connection = sqlite3.connect(path)
        print("Connection to SQLite DB successful")
    except Error as e:
        print(f"The error '{e}' occurred")
    
    return connection

def execute_query(connection, query):
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        connection.commit()
        print("Query executed successfully")
    except Error as e:
        print(f"The error '{e}' occurred")

def execute_read_query(connection, query):
    cursor = connection.cursor()
    result = None
    try:
        cursor.execute(query)
        result = cursor.fetchall()
        return result
    except Error as e:
        print(f"The error '{e}' occurred")

Look at the data before and after the join to validate your data.